# Testing in Python

# Agenda

* Testing pyramid
* Test runner characteristics
* Test structure
* Tests best practices
* Unittest
* Pytest
* RobotFramework

# Testing pyramid



![test_pyramid.png](images/test_pyramid.png)

# Testing pyramid vs Runners



![test_pyramid.png](images/test_pyramid2.png)

# Test runner characteristics

1. **Structure**: test case structure, test naming, config files

2. **Launcher**: run parameters, command line or CI/CD

3. **Setup/teardown and fixtures**: scope, parameters, fixtures

4. **Parametrization**: direct, indirect

5. **Asserts**: flexible, informative

6. **Reporting**: single file, available online

7. **Userful features**: marks/tags/keywords, mocks

8. **Popular methodolodies**: data driven testing, business driven testing, keyword driven testing

9. **Plugins**: additional capabilities (e.g. reruns)

10. **Concurrent execution**

# Test structure

### Arrange -> Act -> Assert = Given -> When -> Then

* **Arrange (Given)**
    * setup method, fixture, factory pattern

* **Act (When)**
    * one simple action

* **Assert (Then)**
    * one or several (not many) asserts that check the action above

* **Cleanup (optional)**
    * for convenience, not precondition!

# Tests best practices

* ***Tests MUST be independent***

* Tests must be informative
    * name
    * assert

* Tests must be simple
    * check one thing at a time

* Tests must not have:
    * inner logic (ifs, loops...)
    * access by indexes

* Tests must be easily readable
    * follow Arrange/Act/Assert (Given/When/Then) approach
    * group steps in needed

# Unittest

* a spiritual successor of JUnit

* built around **unittest.TestCase** class

* can be run from the module, but preferably to be run from a separate file

* tests are methods in a class and must start with test_

# A simple example

In [ ]:
import unittest

class SampleTestCase(unittest.TestCase):                      # Must inherit from unittest.TestCase, any class name
     
    def test_first_test(self):                                # Must start with test_
        number = 42
        self.assertEqual(number, 42)
    
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)  # Just for Jupyter Notebook
    # unittest.main()                                         # Please, use this instead

# test_filename vs filename_test

In [ ]:
!dir unittest_folder

# Writing unit tests for a module

In [ ]:
# unittest_folder/some_code.py

def get_square(number):
    square = number ** 2
    
    return square

In [ ]:
# unittestfolder/test_some_code.py

import unittest
import some_code

class TestSomeCode(unittest.TestCase):
    
    def test_square_positive_number(self):
        square = some_code.get_square(2)
        
        self.assertEqual(square, 4)
    
if __name__ == '__main__':
    unittest.main()

In [ ]:
!py unittest_folder/test_some_code.py

# unittest is a runner

In [ ]:
!py -m unittest --help

# Verbose mode

In [ ]:
!py -m unittest discover -v -s unittest_folder

# Setup and teardown for a test/a class

In [ ]:
import unittest

class SampleTestCase(unittest.TestCase):
     
    def setUp(self):                                          # Must be cameCase
        print("This is a test setup.")
        self.number1 = 42
    
    def tearDown(self):                                       # Must be cameCase
        print("This is a test teardown.")
        self.number1 = 0
        
    @classmethod                                              # Must have this decorator
    def setUpClass(cls):                                      # Must be cameCase
        print("This is a class setup.")
        
    @classmethod                                              # Must have this decorator
    def tearDownClass(cls):                                   # Must be cameCase
        print("This is a class teardown.")

    def test_answer_is_42(self):
        print("This is Test #1")
        self.assertEqual(self.number1, 42)
        
    def test_answer_is_not_666(self):
        print("This is Test #2")
        self.assertNotEqual(self.number1, 666)
    
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)  # Just for Jupyter Notebook

# Failed tests and skipped tests

In [ ]:
import unittest

class SampleTestCase(unittest.TestCase):
     
    def setUp(self):
        self.number1 = 42

    def test_answer_is_666(self):
        self.assertEqual(self.number1, 666)
        
    @unittest.skip("I don't like this test")
    def test_answer_is_not_666(self):
        number2 = 666
        self.assertNotEqual(self.number1, number2)
    
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)  # Just for Jupyter Notebook
    # unittest.main()                                         # Please, use this instead

# Unittest summary



* **Advantages**:
    * in-build Python module, i.e. no installation needed
    * setup/teardown fixtures for tests/classes are supported
    * easy to write and to start
    * mocks are available

* **Disadvantages**:
    * no test/fixture parametrization
    * no data driven testing
    * no reports out-of-the-box

# Pytest



* A third-party solution, developed since 2007

* Can be run via commandline or from IDE

* A test is any function that starts with test_ in a file that starts with test_

* Flexible, customizable, well documented and supported

# Pytest is a runner

In [ ]:
# pip install pytest
!pytest --help

# Simple test

In [ ]:
#pytest/SimpleTest/test_pytest_simple.py

import requests
from hamcrest import assert_that, equal_to

def test_google_is_ok():
    response = requests.get("https://www.google.com")
    
    assert_that(response.status_code, equal_to(200))

In [ ]:
!pytest -v pytest/SimpleTest/

# Parametrization

In [ ]:
# pytest/Parametrization/test_pytest_parametrization.py

import requests
import pytest

from hamcrest import assert_that, equal_to

@pytest.mark.parametrize("url", ["https://www.google.com", "https://www.facebook.com"])
def test_website_is_ok(url):
    response = requests.get(url)
    
    assert_that(response.status_code, equal_to(200))

In [ ]:
!pytest -v pytest/Parametrization/

# Conftest and fixtures

In [ ]:
# pytest/Conftest/conftest.py

import random
import pytest


@pytest.fixture
def random_url():
    random_website = random.choice(["google", "HELLO WORLD"])
    
    return f"https://www.{random_website}.com"

In [ ]:
# pytest/Conftest/test_random_url.py

import requests
from hamcrest import assert_that, equal_to

def test_random_website_is_ok(random_url):
    response = requests.get(random_url)
    
    assert_that(response.status_code, equal_to(200))

In [ ]:
!pytest -v --tb=line pytest/Conftest/

# Setup/teardown fixtures + autouse + scope

In [ ]:
# pytest/SetupTeardown/conftest.py

import random
import pytest


@pytest.fixture(autouse=True)
def setup():
    print("\nThis is the setup fixture")
    

@pytest.fixture(autouse=True)
def clean_up():
    yield
    print("\nThis is the teardown fixture")
    
    
@pytest.fixture(autouse=True, scope="session")
def setup_session():
    print("\nThis is the session setup fixture")
    

@pytest.fixture(autouse=True, scope="session")
def clean_up_session():
    yield
    print("\nThis is the session teardown fixture")

In [ ]:
!pytest -v -s pytest/SetupTeardown/

# Marks vs keywords

* A ***mark*** is a special decorator that we add to a test

* A ***keyword*** is a part of the test name

* Both be used as arguments when starting pytest

* You can look for NOT mark/keyword

In [ ]:
# pytest/MarksKeywords/test/marks/keywords.py 

import pytest 
import requests

from requests.exceptions import ConnectionError
from hamcrest import assert_that, equal_to, is_in

@pytest.mark.foobar                                               # A mark with a custom name
def test_google_is_ok():
    response = requests.get("https://www.google.com")
    
    assert_that(response.status_code, equal_to(200))
    
    
def test_facebook_is_ok():                                       # is_ok is repeated in several tests
    response = requests.get("https://www.facebook.com")
    
    assert_that(response.status_code, equal_to(200))
    

def test_hello_world_is_not_ok():
    try:
        response = requests.get("https://www.hello world.com")
    except ConnectionError:
        pass

In [ ]:
!pytest -v -m foobar pytest/MarksKeywords/

In [ ]:
!pytest -v -k is_ok pytest/MarksKeywords/

In [ ]:
!pytest -v -m "not foobar" pytest/MarksKeywords/

# Html report

* Install with `pip install pytest-html`


* Provides all relevant metrics, can contain a screenshot


* Highly customizable via html

* Can be reduced to a single file

In [ ]:
!pytest pytest/HTMLreport --tb=short --html=pytest/HTMLreport/report.html --self-contained-html & pytest\HTMLreport\report.html

In [ ]:
!pytest pytest/HTMLreport --tb=short --html=pytest/HTMLreport/report.html --css=pytest/HTMLreport/custom_style.css --self-contained-html & pytest\HTMLreport\report.html

# Customized html report 

![html_report.png](images/html_report.png)

# Pytest summary

* **Advantages**
    * supports ALL options of a good runner
    * highly customizable due to plugins (e.g. reruns)
    * has a way to report results
    * good documentation and community

* **Disadvantages**
    * none

# Robot Framework

* developed since 2008 (a continuation of a PhD by Pekka Klärck from 2005)

* designed to facilitate acceptance testing driven development (ATDD)

* focuses on keyword driven development (KDD)

* uses tablular form of code organization

* is based on Python, but offers own syntax

# Robot Framework IDEs?

* no officially supported/created IDE

* **RIDE (RobotFramework IDE)**
    * a community driven project
    * no longer supported

* **RED (RobotFramework EDitor)**
    * a plugin for Eclipse IDE
    * requires additional software and dependencies

* **PyCharm + IntelliBot/Hyper RobotFramework Support**
    * a plugin for PyCharm
    * the most optimal solution

* Neither IDE supports:
    * ***breakpoints***
    * running tests from IDE

# Simple test

In [ ]:
*** Settings ***

*** Variables ***
${MY_TEXT}    Hello world

*** Test Cases ***
Simple Test
    [Documentation]    This is a sample test
    Show My Text    ${MY_TEXT}

*** Keywords ***
Show My Text
    [Arguments]    ${text}
    Log    ${text}

# Setup/teardown for a test/suite

In [ ]:
*** Settings ***
Suite Setup    Prepare Everything
Test Setup    Prepare Something
Test Teardown    Clean Up Something
Suite Teardown    Clean Up Everything

*** Variables ***
${MY_TEXT}    Hello world

*** Test Cases ***
Simple Test
    [Documentation]    This is a sample test
    Show My Text    ${MY_TEXT}

*** Keywords ***
Show My Text
    [Arguments]    ${text}
    Log    ${text}
    
Prepare Everything
    [Documentation] This is a suite setup
    Log    This is a suite setup
    
Prepare Something
    [Documentation] This is a test setup
    Log    This is a test setup

Clean Up Something
    [Documentation] This is a test teardown
    Log    This is a test teardown

Clean Up Everything
    [Documentation] This is a suite teardown
    Log    This is a suite teardown


# Tags

In [ ]:
*** Settings ***

*** Variables ***
${MY_TEXT}    Hello world
${MY_TEXT2}    Bye world

*** Test Cases ***
Simple Test
    [Documentation]    This is a sample test
    [Tags]    regression
    Show My Text    ${MY_TEXT}
    
Simple Test Two
    [Documentation]    This is a sample test two
    [Tags]    smoke    regression
    Show My Text    ${MY_TEXT2}

*** Keywords ***
Show My Text
    [Arguments]    ${text}
    Log    ${text}

In [ ]:
robot -i smoke

In [ ]:
robot -e regression

# Reporting

![robotframework_report.png](images/robotframework_report.png)

# Robot Framework summary

* **Advantages**
    * nice looking and informative reporting out-of-the-box
    * easy-to-understand test code for people without coding background

* **Disadvantages**
    * an uncalled for wrapper for Python
    * no way to set a breakpoint
    * does everything that others do, but more difficult
    * no dedicated IDE
    * doesn't live up to its expectations

# Questions :)

# Thank you for your attention!